<a href="https://colab.research.google.com/github/carolinaenriqz/Optimization-Algorithms-for-Uncertain-Knapsack-Problem/blob/main/ExperimentosComputacionalesSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and Reading Instances

In [1]:
!pip -q install XlsxWriter
!pip install mip

!git clone --depth 1 https://github.com/likr/kplib.git

import os
import pandas as pd
from IPython.display import display
from google.colab import files
import time
import numpy as np
from mip import Model, CONTINUOUS, MAXIMIZE, CBC, OptimizationStatus
from scipy.stats import t as student_t


BASE_DIR = "/content/kplib/00Uncorrelated"
n_values = ["n00050", "n00100", "n00200", "n00500"]
instance_type = "R01000"

instances = []
for n in n_values:
    path = os.path.join(BASE_DIR, n, instance_type)
    if not os.path.isdir(path):
        print(f"Path not found: {path}")
        continue

    files_list = sorted([f for f in os.listdir(path) if f.lower().endswith('.kp')])[:20]

    for file in files_list:
        file_path = os.path.join(path, file)
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()

            num_items = int(lines[1].strip())
            capacity = int(lines[2].strip())
            items = [tuple(map(int, line.strip().split()))
                     for line in lines[4:] if line.strip()]

            if len(items) != num_items:
                print(f"Warning: in {file} {num_items} items were expected but {len(items)} were read")
                continue

            values, weights = zip(*items)
            instances.append({
                "n": num_items,
                "Capacity": capacity,
                "Utilities": list(values),
                "Weights": list(weights),
                "File": file
            })
        except Exception as e:
            print(f"Error reading {file}: {e}")

df_kplib = pd.DataFrame(instances)
print(f"Loaded instances: {df_kplib.shape}")
df_kplib = df_kplib[["File", "n", "Capacity", "Utilities", "Weights"]]
display(df_kplib.head(10))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.1/484.1 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.0 MB/s eta 0:00:00
  Created wheel for cffi: filename=cffi-1.15.0-cp312-cp312-linux_x86_64.whl size=400939 sha256=8c6843b2fd9cbf8bb9e5087bd17f45eaf382ff62d78a51bb0b106492f950271a
  Stored in directory: /root/.cache/pip/wheels/b9/d6/15/0950847bf7d74ea5f0380b8b23a1d81b45bdf48488b4b8237a
Successfully built cffi
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.2 requires cffi>=1.17.0, but you have cffi 1.15.0 which is incompatible.
rpy2 3.5.17 requires 

,File,n,Capacity,Utilities,Weights
0,s000.kp,50,14778,"[845, 758, 421, 259, 512, 405, 784, 304, 477, ...","[804, 448, 81, 321, 508, 933, 110, 552, 707, 5..."
1,s001.kp,50,13598,"[135, 848, 764, 256, 496, 450, 652, 789, 94, 2...","[304, 588, 883, 847, 506, 590, 35, 243, 798, 4..."
2,s002.kp,50,13810,"[957, 948, 57, 85, 836, 736, 670, 309, 606, 60...","[491, 925, 501, 832, 354, 883, 900, 462, 568, ..."
3,s003.kp,50,13596,"[238, 545, 370, 604, 626, 66, 14, 838, 260, 23...","[682, 929, 857, 991, 672, 164, 861, 965, 905, ..."
4,s004.kp,50,11618,"[237, 104, 397, 155, 67, 402, 918, 801, 766, 2...","[281, 535, 472, 343, 998, 196, 413, 203, 633, ..."
5,s005.kp,50,11922,"[623, 742, 796, 943, 740, 923, 30, 466, 944, 6...","[338, 310, 819, 481, 316, 482, 705, 58, 976, 2..."
6,s006.kp,50,12044,"[794, 822, 486, 262, 1, 663, 471, 760, 374, 77...","[881, 526, 117, 663, 313, 197, 484, 139, 210, ..."
7,s007.kp,50,12828,"[324, 151, 651, 73, 536, 366, 58, 508, 38, 434...","[981, 119, 419, 758, 152, 489, 40, 669, 765, 5..."
8,s008.kp,50,10985,"[227, 963, 127, 705, 86, 248, 1000, 210, 642, ...","[703, 950, 844, 504, 198, 151, 529, 510, 72, 9..."
9,s009.kp,50,11487,"[464, 374, 139, 867, 7, 503, 899, 81, 555, 617...","[84, 540, 18, 85, 497, 921, 421, 399, 639, 94,..."


# SP Solver Function (`solve_sp`)

In [2]:
seed = 123
n_smpl = 8
sizes = (50, 100, 200, 500)
k = 20

def solve_sp(bar_c, weights, capacity, pct_deviation, split, t, n_smpl, seed=123, verbose=False):
    if isinstance(split, (list, tuple)) and len(split) == 4 and np.allclose(split, [0.0, 0.5, 0.25, 0.25]):
        if t == 1:
            split = [1.0, 0.0, 0.0, 0.0]
        elif t == 2:
            split = [0.5, 0.5, 0.0, 0.0]
        elif t == 3:
            split = [0.25, 0.5, 0.25, 0.0]
        else:  # t == 4
            split = [0.0, 0.5, 0.25, 0.25]
    pct_x1, pct_x2, pct_x3, pct_x4 = split
    solver_name = CBC
    time_limit = 360000000
    n = len(bar_c)
    n_x1, n_x2, n_x3 = (int(round(pct_x1*n)), int(round(pct_x2*n)), int(round(pct_x3*n))); n_x4 = n - (n_x1 + n_x2 + n_x3)
    c1, c2, c3, c4 = (np.array(bar_c[:n_x1], float), np.array(bar_c[n_x1:n_x1+n_x2], float), np.array(bar_c[n_x1+n_x2:n_x1+n_x2+n_x3], float), np.array(bar_c[n_x1+n_x2+n_x3:n], float))
    w1, w2, w3, w4 = (np.array(weights[:n_x1], float), np.array(weights[n_x1:n_x1+n_x2], float), np.array(weights[n_x1+n_x2:n_x1+n_x2+n_x3], float), np.array(weights[n_x1+n_x2+n_x3:n], float))
    rng = np.random.default_rng(seed)
    def _lh(v, delta):
        low, high = v*(1-delta), v*(1+delta)
        return np.minimum(low, high), np.maximum(low, high)
    low2, high2 = _lh(c2, pct_deviation); U2_base = rng.uniform(low2, high2, size=(n_smpl, n_x2))
    low3, high3 = _lh(c3, pct_deviation); U3_base = rng.uniform(low3, high3, size=(n_smpl, n_x3))
    low4, high4 = _lh(c4, pct_deviation); U4_base = rng.uniform(low4, high4, size=(n_smpl, n_x4))
    mu2 = U2_base.mean(axis=0) if U2_base.size else np.array([], float)
    mu3 = U3_base.mean(axis=0) if U3_base.size else np.array([], float)
    mu4 = U4_base.mean(axis=0) if U4_base.size else np.array([], float)
    tile3 = np.tile(mu3, (n_smpl, 1)) if mu3.size else np.zeros((n_smpl, 0), float)
    tile4 = np.tile(mu4, (n_smpl, 1)) if mu4.size else np.zeros((n_smpl, 0), float)
    tile3 = np.tile(c3, (n_smpl, 1)) if c3.size else np.zeros((n_smpl, 0), float)
    tile4 = np.tile(c4, (n_smpl, 1)) if c4.size else np.zeros((n_smpl, 0), float)
    S2 = S3 = S4 = n_smpl
    p2 = p3 = p4 = 1.0 / n_smpl

    # ---------- t == 1 ----------
    if t == 1:
        mean2, mean3, mean4 = mu2, mu3, mu4
        c_mean = np.concatenate([c1, mean2, mean3, mean4]).astype(float)
        w_all = np.concatenate([w1, w2, w3, w4]).astype(float)
        m = Model(sense=MAXIMIZE, solver_name=solver_name); m.max_seconds = time_limit
        x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x_{j}") for j in range(n)]
        m.objective = sum(c_mean[j] * x[j] for j in range(n))
        m += (sum(w_all[j] * x[j] for j in range(n)) <= capacity), "cap"
        t0 = time.perf_counter(); status = m.optimize(); t1 = time.perf_counter()
        solved   = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val  = m.objective_value if solved else None
        x_opt    = np.array([var.x for var in x]) if solved else None
        if verbose:
            print(f"[MIP] SP1 | status={status} | obj={obj_val} | vars/cons={m.num_cols}/{m.num_rows} | time={t1-t0:.2f}s")
        assert m.num_cols == n and m.num_rows == 1, f"Expected Vars/Cons {n}/1, got {m.num_cols}/{m.num_rows}"
        return {"t": 1, "status": str(status), "objective": obj_val, "n_vars": m.num_cols, "n_constrs": m.num_rows, "x": x_opt, "model": m, "split": split}

    # ---------- t == 2 ----------
    elif t == 2:
        c234 = np.concatenate([c2, c3, c4]).astype(float) if (n_x2+n_x3+n_x4)>0 else np.array([], float)
        w2_u = np.concatenate([w2, w3, w4]).astype(float)  if (n_x2+n_x3+n_x4)>0 else np.array([], float)
        U2 = np.concatenate([U2_base, tile3, tile4], axis=1) if c234.size>0 else np.zeros((n_smpl, 0), float)
        n_x2_u = c234.size
        S2 = n_smpl; p2 = 1.0 / n_smpl
        m2 = Model(sense=MAXIMIZE, solver_name=solver_name); m2.max_seconds = time_limit
        x1 = [m2.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m2.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}") for s2 in range(S2) for j in range(n_x2_u)}
        obj = 0.0
        for j in range(n_x1): obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2_u):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        m2.objective = obj
        for s2 in range(S2):
            expr = 0.0
            for j in range(n_x1):    expr += w1[j]  * x1[j]
            for j in range(n_x2_u):  expr += w2_u[j] * x2[(s2, j)]
            m2 += (expr <= capacity), f"cap_{s2}"

        t0 = time.perf_counter(); status = m2.optimize(); t1 = time.perf_counter()
        solved = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m2.objective_value if solved else None

        if verbose:
            print(f"SP2 | status={status} | obj={obj_val} | vars/cons={m2.num_cols}/{m2.num_rows} | time={t1-t0:.2f}s")
        assert m2.num_cols == n_x1 + n_x2_u*S2, "Unexpected variable count in SP2"
        assert m2.num_rows == S2,               "Unexpected constraint count in SP2"

        return {"t": 2, "status": str(status), "objective": obj_val, "n_vars": m2.num_cols, "n_constrs": m2.num_rows, "model": m2, "split": split}

    # ---------- t == 3 ----------
    elif t == 3:
        c34 = np.concatenate([c3, c4]).astype(float) if (n_x3+n_x4)>0 else np.array([], float)
        w3_u = np.concatenate([w3, w4]).astype(float)  if (n_x3+n_x4)>0 else np.array([], float)

        U2  = U2_base
        U34 = np.concatenate([U3_base, tile4], axis=1) if c34.size>0 else np.zeros((n_smpl, 0), float)
        n_x3_u = c34.size
        S2 = S3 = n_smpl; p2 = p3 = 1.0 / n_smpl
        m3 = Model(sense=MAXIMIZE, solver_name=solver_name); m3.max_seconds = time_limit
        x1 = [m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}") for s2 in range(S2) for j in range(n_x2)}
        x3 = {(s2, s3, k): m3.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x3_{s2}_{s3}_{k}") for s2 in range(S2) for s3 in range(S3) for k in range(n_x3_u)}

        obj = 0.0
        for j in range(n_x1): obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        for s2 in range(S2):
            for s3 in range(S3):
                for k in range(n_x3_u):
                    obj += (p2 * p3 * U34[s3, k]) * x3[(s2, s3, k)]
        m3.objective = obj

        for s2 in range(S2):
            for s3 in range(S3):
                expr = 0.0
                for j in range(n_x1):    expr += w1[j]   * x1[j]
                for j in range(n_x2):    expr += w2[j]   * x2[(s2, j)]
                for k in range(n_x3_u):  expr += w3_u[k] * x3[(s2, s3, k)]
                m3 += (expr <= capacity), f"cap_{s2}_{s3}"

        t0 = time.perf_counter(); status = m3.optimize(); t1 = time.perf_counter()
        solved = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m3.objective_value if solved else None

        if verbose:
            print(f"SP3 | status={status} | obj={obj_val} | vars/cons={m3.num_cols}/{m3.num_rows} | time={t1-t0:.2f}s")
        assert m3.num_cols == n_x1 + n_x2*(S2) + n_x3_u*(S2*S3), "Unexpected variable count in SP3"
        assert m3.num_rows == S2*S3,                              "Unexpected constraint count in SP3"

        return {"t": 3, "status": str(status), "objective": obj_val, "n_vars": m3.num_cols, "n_constrs": m3.num_rows, "model": m3, "split": split}

    # ---------- t == 4 ----------
    else:
        U2, U3, U4 = U2_base, U3_base, U4_base
        S2 = S3 = S4 = n_smpl
        p2 = p3 = p4 = 1.0 / n_smpl

        m4 = Model(sense=MAXIMIZE, solver_name=solver_name); m4.max_seconds = time_limit
        x1 = [m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x1_{j}") for j in range(n_x1)]
        x2 = {(s2, j): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x2_{s2}_{j}")
              for s2 in range(S2) for j in range(n_x2)}
        x3 = {(s2, s3, k): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x3_{s2}_{s3}_{k}")
              for s2 in range(S2) for s3 in range(S3) for k in range(n_x3)}
        x4 = {(s2, s3, s4, h): m4.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name=f"x4_{s2}_{s3}_{s4}_{h}")
              for s2 in range(S2) for s3 in range(S3) for s4 in range(S4) for h in range(n_x4)}
        obj = 0.0
        for j in range(n_x1):
            obj += c1[j] * x1[j]
        for s2 in range(S2):
            for j in range(n_x2):
                obj += (p2 * U2[s2, j]) * x2[(s2, j)]
        for s2 in range(S2):
            for s3 in range(S3):
                for k in range(n_x3):
                    obj += (p2 * p3 * U3[s3, k]) * x3[(s2, s3, k)]
        for s2 in range(S2):
            for s3 in range(S3):
                for s4 in range(S4):
                    for h in range(n_x4):
                        obj += (p2 * p3 * p4 * U4[s4, h]) * x4[(s2, s3, s4, h)]
        m4.objective = obj
        for s2 in range(S2):
            for s3 in range(S3):
                for s4 in range(S4):
                    expr = 0.0
                    for j in range(n_x1): expr += w1[j] * x1[j]
                    for j in range(n_x2): expr += w2[j] * x2[(s2, j)]
                    for k in range(n_x3): expr += w3[k] * x3[(s2, s3, k)]
                    for h in range(n_x4): expr += w4[h] * x4[(s2, s3, s4, h)]
                    m4 += (expr <= capacity), f"cap_{s2}_{s3}_{s4}"
        t0 = time.perf_counter(); status = m4.optimize(); t1 = time.perf_counter()  # timing
        solved  = status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE)
        obj_val = m4.objective_value if solved else None
        if verbose:
            print(f"SP4 | status={status} | obj={obj_val} | vars/cons={m4.num_cols}/{m4.num_rows} | time={t1-t0:.2f}s")
        exp_vars = n_x1 + n_x2*S2 + n_x3*(S2*S3) + n_x4*(S2*S3*S4)
        exp_cons = S2*S3*S4
        assert m4.num_cols == exp_vars, f"Expected Vars {exp_vars}, got {m4.num_cols}"
        assert m4.num_rows == exp_cons, f"Expected Cons {exp_cons}, got {m4.num_rows}"
        return {"t": 4, "status": str(status), "objective": obj_val, "n_vars": m4.num_cols, "n_constrs": m4.num_rows, "model": m4, "split": split }


# Processing SP Results

In [3]:
rows = []

split_A = [0.00, 0.50, 0.25, 0.25]  # SP1..SP4
split_B = [0.25, 0.75, 0.00, 0.00]  # only SP2 -> SP2split_75
split_C = [0.50, 0.50, 0.00, 0.00]  # only SP2 -> SP2split_50
split_D = [0.75, 0.25, 0.00, 0.00]  # only SP2 -> SP2split_25

def label_for(t, split_list):
    """Label consistent with your semantics; None if not applicable."""
    if split_list == split_A:
        return {1:"SP1", 2:"SP2", 3:"SP3", 4:"SP4"}.get(t, None)
    if t != 2:
        return None
    if split_list == split_B: return "SP2split_75"
    if split_list == split_C: return "SP2split_50"
    if split_list == split_D: return "SP2split_25"
    return None

for pct_deviation in (0.1, 0.5, 1.0, 1.5, 2.0):
    for n_target in sizes:
        df_sub = df_kplib[df_kplib["n"] == n_target].head(k)
        for idx, row in df_sub.iterrows():
            bar_c = np.array(row["Utilities"], float)
            weights = np.array(row["Weights"], float)
            cap   = float(row["Capacity"])
            arc   = row["File"] if "File" in df_kplib.columns else f"idx_{idx}"
            assert bar_c.size == weights.size, f"Dim mismatch in {arc}"

            # --- Nominal ---
            m_nom = Model(sense=MAXIMIZE, solver_name=CBC)
            x_nom = [m_nom.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0) for _ in range(len(bar_c))]
            m_nom.objective = sum(bar_c[j]*x_nom[j] for j in range(len(bar_c)))
            m_nom += (sum(weights[j]*x_nom[j] for j in range(len(bar_c))) <= cap)
            m_nom.optimize()
            rows.append({
                "File": arc, "n": int(row["n"]), "Capacity": cap, "%Dev": 0.0,
                "t": 0, "split": "nominal", "objective": round(float(m_nom.objective_value), 6),
                "time_s": None, "label": "Nominal"
            })

            # --- SP1..SP4 with split_A + SP2_splits (B/C/D) ---
            for t in (1, 2, 3, 4):
                for split in (split_A, split_B, split_C, split_D):
                    if split in (split_B, split_C, split_D) and t != 2:
                        continue
                    t0 = time.perf_counter()
                    sol = solve_sp(bar_c, weights, cap, pct_deviation, split, t=t,
                                   n_smpl=n_smpl, seed=seed, verbose=True)
                    t1 = time.perf_counter()
                    obj = float(sol["objective"]) if sol["objective"] is not None else np.nan
                    split_str = "-".join(f"{p:.2f}" for p in sol["split"])
                    rows.append({
                        "File": arc, "n": int(row["n"]), "Capacity": cap, "%Dev": pct_deviation,
                        "t": t, "split": split_str, "objective": round(obj, 6),
                        "time_s": round(t1 - t0, 6), "label": label_for(t, split)
                    })


df_long = pd.DataFrame(rows)

# Nominal per instance (does not depend on %Dev)
nominal = (df_long[df_long["label"]=="Nominal"][["File","n","Capacity","objective"]]
           .drop_duplicates(subset=["File","n","Capacity"])
           .rename(columns={"objective":"Nominal"}))

df_long = df_long[df_long["label"].notna()].copy()
df_long = df_long.merge(nominal, on=["File","n","Capacity"])
df_long["inc_vs_sp"] = (df_long["objective"] - df_long["Nominal"]) / df_long["Nominal"]
df_long


[MIP] SP1 | status=OptimizationStatus.OPTIMAL | obj=21047.483258928572 | vars/cons=50/1 | time=0.00s
SP2 | status=OptimizationStatus.OPTIMAL | obj=21019.764326007054 | vars/cons=100/3 | time=0.01s
SP2 | status=OptimizationStatus.OPTIMAL | obj=21066.422057604228 | vars/cons=126/3 | time=0.00s
SP2 | status=OptimizationStatus.OPTIMAL | obj=21019.764326007054 | vars/cons=100/3 | time=0.01s
SP2 | status=OptimizationStatus.OPTIMAL | obj=21045.09342342331 | vars/cons=74/3 | time=0.01s
SP3 | status=OptimizationStatus.OPTIMAL | obj=21084.66356683191 | vars/cons=204/9 | time=0.02s
SP4 | status=OptimizationStatus.OPTIMAL | obj=20938.3953644224 | vars/cons=534/27 | time=0.01s
[MIP] SP1 | status=OptimizationStatus.OPTIMAL | obj=19845.257575757576 | vars/cons=50/1 | time=0.00s
SP2 | status=OptimizationStatus.OPTIMAL | obj=19715.481213280716 | vars/cons=100/3 | time=0.00s
SP2 | status=OptimizationStatus.OPTIMAL | obj=19918.87935499067 | vars/cons=126/3 | time=0.00s
SP2 | status=OptimizationStatus.OPT

,File,n,Capacity,%Dev,t,split,objective,time_s,label,Nominal,inc_vs_sp
0,s000.kp,50,14778.0,0.0,0,nominal,21047.483259,NaN,Nominal,21047.483259,0.000000
1,s000.kp,50,14778.0,0.1,1,1.00-0.00-0.00-0.00,21047.483259,0.021359,SP1,21047.483259,0.000000
2,s000.kp,50,14778.0,0.1,2,0.50-0.50-0.00-0.00,21019.764326,0.021112,SP2,21047.483259,-0.001317
3,s000.kp,50,14778.0,0.1,2,0.25-0.75-0.00-0.00,21066.422058,0.013265,SP2split_75,21047.483259,0.000900
4,s000.kp,50,14778.0,0.1,2,0.50-0.50-0.00-0.00,21019.764326,0.018698,SP2split_50,21047.483259,-0.001317
...,...,...,...,...,...,...,...,...,...,...,...
315,s001.kp,500,131012.0,2.0,2,0.25-0.75-0.00-0.00,253639.856613,0.072150,SP2split_75,202855.782427,0.250346
316,s001.kp,500,131012.0,2.0,2,0.50-0.50-0.00-0.00,235083.739727,0.084618,SP2split_50,202855.782427,0.158871
317,s001.kp,500,131012.0,2.0,2,0.75-0.25-0.00-0.00,216637.294789,0.055823,SP2split_25,202855.782427,0.067937
318,s001.kp,500,131012.0,2.0,3,0.25-0.50-0.25-0.00,256348.541347,0.165717,SP3,202855.782427,0.263698


# Aggregating Results

In [4]:
def iqr(x):
    x = np.asarray(x, dtype=float)
    if x.size < 2: return np.nan
    return np.percentile(x, 75) - np.percentile(x, 25)

def ci95(x):
    """Return (low, high) of the 95% CI for the mean."""
    x = np.asarray(x, dtype=float)
    m = x.size
    if m < 2: return (np.nan, np.nan)
    mean = np.mean(x)
    se   = np.std(x, ddof=1) / np.sqrt(m)
    tcrit = student_t.ppf(0.975, df=m-1)
    h = tcrit * se
    return (mean - h, mean + h)



# ---------- aggregate: ALL together (increment + objective + time) ----------
agg_all = (
    df_long
    .groupby(["n", "%Dev", "label"])
    .agg(
        count       = ("inc_vs_sp", "size"),

        # increment vs SP
        inc_mean    = ("inc_vs_sp", "mean"),
        inc_median  = ("inc_vs_sp", "median"),
        inc_std     = ("inc_vs_sp", "std"),
        inc_iqr     = ("inc_vs_sp", iqr),
        inc_ci_low  = ("inc_vs_sp", lambda x: ci95(x)[0]),
        inc_ci_high = ("inc_vs_sp", lambda x: ci95(x)[1]),

        # absolute objective
        obj_mean    = ("objective", "mean"),
        obj_median  = ("objective", "median"),
        obj_std     = ("objective", "std"),
        obj_iqr     = ("objective", iqr),
        obj_ci_low  = ("objective", lambda x: ci95(x)[0]),
        obj_ci_high = ("objective", lambda x: ci95(x)[1]),

        # time
        time_mean    = ("time_s", "mean"),
        time_median  = ("time_s", "median"),
        time_std     = ("time_s", "std"),
        time_iqr     = ("time_s", iqr),
        time_ci_low  = ("time_s", lambda x: ci95(x)[0]),
        time_ci_high = ("time_s", lambda x: ci95(x)[1]),
    )
    .reset_index()
)

agg_all = agg_all[agg_all["label"].isin(["SP1", "SP2", "SP3", "SP4", "SP2split_25", "SP2split_50", "SP2split_75"])]

# Fixed classification of uncertainty levels
def classify_uncertainty(x):
    if x == 0.1:
        return "Very low"
    elif x == 0.5:
        return "Low"
    elif x == 1.0:
        return "Medium"
    elif x == 1.5:
        return "High"
    elif x == 2.0:
        return "Very high"
    else:
        return "Unclassified"

agg_all["Uncertainty"] = agg_all["%Dev"].apply(classify_uncertainty)
for c in [col for col in agg_all.columns if any(s in col for s in ("_mean","_median","_std","_iqr","_ci_low","_ci_high"))]:
    agg_all[c] = agg_all[c].astype(float).round(6)

# ---------- derived: INC only, OBJECTIVE only, and TIME only ----------
cols_keys = ["n","Uncertainty","label"]

agg_inc  = agg_all[cols_keys + ["inc_mean","inc_ci_low","inc_ci_high", "inc_std", "inc_median","inc_iqr"]].copy()
agg_obj  = agg_all[cols_keys + ["obj_mean","obj_ci_low","obj_ci_high", "obj_std", "obj_median","obj_iqr"]].copy()
agg_time = agg_all[cols_keys + ["time_mean","time_ci_low","time_ci_high", "time_std", "time_median","time_iqr"]].copy()

agg_inc_A  = agg_inc[agg_inc["label"].isin(["SP1", "SP2", "SP3", "SP4"])].copy()
agg_obj_A  = agg_obj[agg_obj["label"].isin(["SP1", "SP2", "SP3", "SP4"])].copy()
agg_time_A = agg_time[agg_time["label"].isin(["SP1", "SP2", "SP3", "SP4"])].copy()

agg_inc_B  = agg_inc[agg_inc["label"].isin(["SP2split_25", "SP2split_50", "SP2split_75"])].copy()
agg_obj_B  = agg_obj[agg_obj["label"].isin(["SP2split_25", "SP2split_50", "SP2split_75"])].copy()
agg_time_B = agg_time[agg_time["label"].isin(["SP2split_25", "SP2split_50", "SP2split_75"])].copy()


try:
    display(agg_all.head(10))
    display(agg_inc.head(10)); display(agg_obj.head(10)); display(agg_time.head(10))
except Exception:
    pass



,n,%Dev,label,count,inc_mean,inc_median,inc_std,inc_iqr,inc_ci_low,inc_ci_high,...,obj_iqr,obj_ci_low,obj_ci_high,time_mean,time_median,time_std,time_iqr,time_ci_low,time_ci_high,Uncertainty
1,50,0.1,SP1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,601.112842,12808.507584,28084.233251,0.013934,0.013934,0.010501,0.007425,-0.080416,0.108283,Very low
2,50,0.1,SP2,2,-0.003928,-0.003928,0.003693,0.002611,-0.037107,0.029251,...,652.141557,12081.378635,28653.866904,0.016152,0.016152,0.007015,0.004960,-0.046878,0.079181,Very low
3,50,0.1,SP2split_25,2,0.001501,0.001501,0.002284,0.001615,-0.019016,0.022019,...,568.998915,13246.277799,27705.911217,0.014630,0.014630,0.010172,0.007192,-0.076760,0.106019,Very low
4,50,0.1,SP2split_50,2,-0.003928,-0.003928,0.003693,0.002611,-0.037107,0.029251,...,652.141557,12081.378635,28653.866904,0.013948,0.013948,0.006717,0.004750,-0.046400,0.074297,Very low
5,50,0.1,SP2split_75,2,0.002305,0.002305,0.001987,0.001405,-0.015547,0.020157,...,573.771351,13202.194442,27783.106971,0.012328,0.012328,0.001326,0.000937,0.000415,0.024240,Very low
6,50,0.1,SP3,2,0.001242,0.001242,0.000742,0.000525,-0.005424,0.007908,...,612.586332,12688.429875,28255.724594,0.034212,0.034212,0.023748,0.016792,-0.179157,0.247580,Very low
7,50,0.1,SP4,2,-0.004576,-0.004576,0.000858,0.000607,-0.012289,0.003137,...,585.950349,12907.239915,27797.650115,0.082910,0.082910,0.024690,0.017458,-0.138921,0.304742,Very low
8,50,0.5,SP1,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,601.112842,12808.507584,28084.233251,0.008050,0.008050,0.002281,0.001613,-0.012445,0.028545,Low
9,50,0.5,SP2,2,-0.007378,-0.007378,0.024802,0.017538,-0.230217,0.215462,...,955.263745,8168.290652,32443.844094,0.015095,0.015095,0.000256,0.000181,0.012795,0.017395,Low
10,50,0.5,SP2split_25,2,0.012610,0.012610,0.004406,0.003116,-0.026979,0.052199,...,544.988161,13777.595491,27627.057784,0.012819,0.012819,0.003230,0.002284,-0.016202,0.041840,Low


,n,Uncertainty,label,inc_mean,inc_ci_low,inc_ci_high,inc_std,inc_median,inc_iqr
1,50,Very low,SP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,50,Very low,SP2,-0.003928,-0.037107,0.029251,0.003693,-0.003928,0.002611
3,50,Very low,SP2split_25,0.001501,-0.019016,0.022019,0.002284,0.001501,0.001615
4,50,Very low,SP2split_50,-0.003928,-0.037107,0.029251,0.003693,-0.003928,0.002611
5,50,Very low,SP2split_75,0.002305,-0.015547,0.020157,0.001987,0.002305,0.001405
6,50,Very low,SP3,0.001242,-0.005424,0.007908,0.000742,0.001242,0.000525
7,50,Very low,SP4,-0.004576,-0.012289,0.003137,0.000858,-0.004576,0.000607
8,50,Low,SP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,50,Low,SP2,-0.007378,-0.230217,0.215462,0.024802,-0.007378,0.017538
10,50,Low,SP2split_25,0.012610,-0.026979,0.052199,0.004406,0.012610,0.003116


,n,Uncertainty,label,obj_mean,obj_ci_low,obj_ci_high,obj_std,obj_median,obj_iqr
1,50,Very low,SP1,20446.370418,12808.507584,28084.233251,850.101933,20446.370418,601.112842
2,50,Very low,SP2,20367.622769,12081.378635,28653.866904,922.267434,20367.622769,652.141557
3,50,Very low,SP2split_25,20476.094508,13246.277799,27705.911217,804.685983,20476.094508,568.998915
4,50,Very low,SP2split_50,20367.622769,12081.378635,28653.866904,922.267434,20367.622769,652.141557
5,50,Very low,SP2split_75,20492.650706,13202.194442,27783.106971,811.435227,20492.650706,573.771351
6,50,Very low,SP3,20472.077234,12688.429875,28255.724594,866.327900,20472.077234,612.586332
7,50,Very low,SP4,20352.445015,12907.239915,27797.650115,828.658930,20352.445015,585.950349
8,50,Low,SP1,20446.370418,12808.507584,28084.233251,850.101933,20446.370418,601.112842
9,50,Low,SP2,20306.067373,8168.290652,32443.844094,1350.946944,20306.067373,955.263745
10,50,Low,SP2split_25,20702.326638,13777.595491,27627.057784,770.729648,20702.326638,544.988161


,n,Uncertainty,label,time_mean,time_ci_low,time_ci_high,time_std,time_median,time_iqr
1,50,Very low,SP1,0.013934,-0.080416,0.108283,0.010501,0.013934,0.007425
2,50,Very low,SP2,0.016152,-0.046878,0.079181,0.007015,0.016152,0.004960
3,50,Very low,SP2split_25,0.014630,-0.076760,0.106019,0.010172,0.014630,0.007192
4,50,Very low,SP2split_50,0.013948,-0.046400,0.074297,0.006717,0.013948,0.004750
5,50,Very low,SP2split_75,0.012328,0.000415,0.024240,0.001326,0.012328,0.000937
6,50,Very low,SP3,0.034212,-0.179157,0.247580,0.023748,0.034212,0.016792
7,50,Very low,SP4,0.082910,-0.138921,0.304742,0.024690,0.082910,0.017458
8,50,Low,SP1,0.008050,-0.012445,0.028545,0.002281,0.008050,0.001613
9,50,Low,SP2,0.015095,0.012795,0.017395,0.000256,0.015095,0.000181
10,50,Low,SP2split_25,0.012819,-0.016202,0.041840,0.003230,0.012819,0.002284


In [7]:
sp_experiment_A = agg_all[agg_all["label"].isin(["SP2", "SP3", "SP4"])].copy()
order = ["Very low", "Low", "Medium", "High", "Very high"]
sp_experiment_A["Uncertainty"] = pd.Categorical(sp_experiment_A["Uncertainty"], categories=order, ordered=True)



# --- 4) Dataset for FIGURE X (fix n=200; columns: Level, Method, mean/low/high) ---
sp_experiment_A_fig_n_200_uncertainty = (
    sp_experiment_A[sp_experiment_A["n"] == 200]
    .loc[:, ["Uncertainty", "label", "inc_mean", "inc_ci_low", "inc_ci_high"]]
    .sort_values(["Uncertainty", "label"])
)



# --- 5B) (Optional) Save to CSV if you prefer ---
sp_experiment_A_fig_n_200_uncertainty_excel = sp_experiment_A_fig_n_200_uncertainty.copy()

# Multiply by 100 and round to 2 decimals in the three columns
cols_pct = ["inc_mean", "inc_ci_low", "inc_ci_high"]
sp_experiment_A_fig_n_200_uncertainty_excel.loc[:, cols_pct] = (
    sp_experiment_A_fig_n_200_uncertainty_excel.loc[:, cols_pct]
    .applymap(lambda x: round(100*x, 2))
)


sp_experiment_B = agg_all[agg_all["label"].isin(["SP2split_25", "SP2split_50", "SP2split_75"])].copy()
sp_experiment_B["Uncertainty"] = sp_experiment_B["%Dev"].apply(classify_uncertainty)
sp_experiment_B["Uncertainty"] = pd.Categorical(sp_experiment_B["Uncertainty"], categories=order, ordered=True)
sp_experiment_B_fig_n_200_uncertainty = (
    sp_experiment_B[sp_experiment_B["n"] == 200]
    .loc[:, ["Uncertainty", "label", "inc_mean", "inc_ci_low", "inc_ci_high"]]
    .sort_values(["Uncertainty", "label"])
)

sp_experiment_B_fig_n_200_uncertainty_excel = sp_experiment_B_fig_n_200_uncertainty.copy()
sp_experiment_B_fig_n_200_uncertainty_excel.loc[:, cols_pct] = (
    sp_experiment_B_fig_n_200_uncertainty_excel.loc[:, cols_pct]
    .applymap(lambda x: round(100*x, 2))
)

display(sp_experiment_A_fig_n_200_uncertainty_excel)
display(sp_experiment_B_fig_n_200_uncertainty_excel)

## Figure 2 -> For each n, and a fixed uncertainty level (High), see how each approach varies
sp_experiment_A_fig_fixed_uncertainty_high = sp_experiment_A[sp_experiment_A["Uncertainty"] == "High"].copy()
sp_experiment_A_fig_fixed_uncertainty_high = sp_experiment_A_fig_fixed_uncertainty_high.loc[:, ["n", "label", "inc_mean", "inc_ci_low", "inc_ci_high"]]
sp_experiment_A_fig_fixed_uncertainty_high_excel = sp_experiment_A_fig_fixed_uncertainty_high.copy()
sp_experiment_A_fig_fixed_uncertainty_high_excel.loc[:, cols_pct] = (
    sp_experiment_A_fig_fixed_uncertainty_high_excel.loc[:, cols_pct]
    .applymap(lambda x: round(100*x, 2))
)
sp_experiment_A_fig_fixed_uncertainty_high_excel

sp_experiment_B_fig_fixed_uncertainty_high = sp_experiment_B[sp_experiment_B["Uncertainty"] == "High"].copy()
sp_experiment_B_fig_fixed_uncertainty_high = sp_experiment_B_fig_fixed_uncertainty_high.loc[:, ["n", "label", "inc_mean", "inc_ci_low", "inc_ci_high"]]
sp_experiment_B_fig_fixed_uncertainty_high_excel = sp_experiment_B_fig_fixed_uncertainty_high.copy()
sp_experiment_B_fig_fixed_uncertainty_high_excel.loc[:, cols_pct] = (
    sp_experiment_B_fig_fixed_uncertainty_high_excel.loc[:, cols_pct]
    .applymap(lambda x: round(100*x, 2))
)
sp_experiment_B_fig_fixed_uncertainty_high_excel

## Figure 3 - Computational time for High Uncertainty level
sp_experiment_A_time_uncertainty_high = sp_experiment_A.copy()
sp_experiment_A_time_uncertainty_high = sp_experiment_A_time_uncertainty_high[sp_experiment_A_time_uncertainty_high["Uncertainty"] == "High"]
sp_experiment_A_time_uncertainty_high = sp_experiment_A_time_uncertainty_high.loc[:, ["n", "label", "time_mean", "time_ci_low", "time_ci_high"]]
sp_experiment_A_time_uncertainty_high


sp_experiment_B_time_uncertainty_high  = sp_experiment_B.copy()
sp_experiment_B_time_uncertainty_high = sp_experiment_B_time_uncertainty_high[sp_experiment_B_time_uncertainty_high["Uncertainty"] == "High"]
sp_experiment_B_time_uncertainty_high = sp_experiment_B_time_uncertainty_high.loc[:, ["n", "label", "time_mean", "time_ci_low", "time_ci_high"]]
sp_experiment_B_time_uncertainty_high


agg_all_no_nominal = agg_all[agg_all["label"].isin(["SP1", "SP2", "SP3", "SP4", "SP2split_25", "SP2split_50", "SP2split_75"])]
agg_all_no_nominal

table_mean_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_mean"].mean().reset_index()
table_mean_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_mean"].mean().reset_index()
table_mean_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_mean"].mean().reset_index()

table_cilow_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_ci_low"].mean().reset_index()
table_cilow_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_ci_low"].mean().reset_index()
table_cilow_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_ci_low"].mean().reset_index()

table_cihigh_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_ci_high"].mean().reset_index()
table_cihigh_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_ci_high"].mean().reset_index()
table_cihigh_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_ci_high"].mean().reset_index()

table_std_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_std"].mean().reset_index()
table_std_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_std"].mean().reset_index()
table_std_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_std"].mean().reset_index()

table_median_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_median"].mean().reset_index()
table_median_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_median"].mean().reset_index()
table_median_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_median"].mean().reset_index()

table_iqr_inc  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["inc_iqr"].mean().reset_index()
table_iqr_obj  = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["obj_iqr"].mean().reset_index()
table_iqr_time = agg_all_no_nominal.groupby(["n", "Uncertainty", "label"])["time_iqr"].mean().reset_index()


/tmp/ipython-input-427169447.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: round(100*x, 2))
/tmp/ipython-input-427169447.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: round(100*x, 2))


,Uncertainty,label,inc_mean,inc_ci_low,inc_ci_high
74,Very low,SP2,0.02,-0.56,0.59
78,Very low,SP3,0.03,-0.64,0.69
79,Very low,SP4,0.13,-0.26,0.51
81,Low,SP2,0.97,-3.97,5.90
85,Low,SP3,1.20,-2.86,5.26
86,Low,SP4,2.40,1.31,3.48
88,Medium,SP2,3.98,-8.03,15.99
92,Medium,SP3,5.60,2.43,8.77
93,Medium,SP4,9.05,5.13,12.97
95,High,SP2,8.67,-6.02,23.35


,Uncertainty,label,inc_mean,inc_ci_low,inc_ci_high
75,Very low,SP2split_25,-0.11,-0.42,0.20
76,Very low,SP2split_50,0.02,-0.56,0.59
77,Very low,SP2split_75,0.04,-3.07,3.15
82,Low,SP2split_25,0.11,-0.39,0.60
83,Low,SP2split_50,0.97,-3.97,5.90
84,Low,SP2split_75,1.60,-15.51,18.70
89,Medium,SP2split_25,1.46,0.18,2.74
90,Medium,SP2split_50,3.98,-8.03,15.99
91,Medium,SP2split_75,6.62,-24.60,37.85
96,High,SP2split_25,3.76,1.89,5.62


/tmp/ipython-input-427169447.py:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: round(100*x, 2))
/tmp/ipython-input-427169447.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: round(100*x, 2))


# Exporting Results

In [9]:
# ---------- export ----------
with pd.ExcelWriter("ExperimentosComputacionalesSP.xlsx") as writer:

  # MAIN TABLES

    agg_inc_A.to_excel(writer, sheet_name="agg_inc_A", index=False)
    agg_obj_A.to_excel(writer, sheet_name="agg_obj_A", index=False)
    agg_time_A.to_excel(writer, sheet_name="agg_time_A", index=False)
    agg_inc_B.to_excel(writer, sheet_name="agg_inc_B", index=False)
    agg_obj_B.to_excel(writer, sheet_name="agg_obj_B", index=False)
    agg_time_B.to_excel(writer, sheet_name="agg_time_B", index=False)

  # FIGURE DATA

    sp_experiment_A_fig_n_200_uncertainty_excel.to_excel(writer, sheet_name="Exp_A_fig_n_fijo_200", index=False)
    sp_experiment_B_fig_n_200_uncertainty_excel.to_excel(writer, sheet_name="Exp_B_fig_n_fijo_200", index=False)

    sp_experiment_A_fig_fixed_uncertainty_high_excel.to_excel(writer, sheet_name="Exp_A_fig_incer_alta", index=False)
    sp_experiment_B_fig_fixed_uncertainty_high_excel.to_excel(writer, sheet_name="Exp_B_fig_incer_alta", index=False)

    sp_experiment_A_time_uncertainty_high.to_excel(writer, sheet_name="Exp_A_tiempo_incer_alta", index=False)
    sp_experiment_B_time_uncertainty_high.to_excel(writer, sheet_name="Exp_B_tiempo_incer_alta", index=False)

  # TABLE DATA

    agg_all.to_excel(writer, sheet_name="all", index=False)
    agg_inc.to_excel(writer, sheet_name="inc", index=False)
    agg_obj.to_excel(writer, sheet_name="obj", index=False)
    agg_time.to_excel(writer, sheet_name="time", index=False)

  # METRICS REPORT

    table_mean_inc.to_excel(writer, sheet_name="tabla_mean_inc", index=False)
    table_mean_obj.to_excel(writer, sheet_name="tabla_mean_obj", index=False)
    table_mean_time.to_excel(writer, sheet_name="tabla_mean_time", index=False)
    table_cilow_inc.to_excel(writer, sheet_name="tabla_cilow_inc", index=False)
    table_cilow_obj.to_excel(writer, sheet_name="tabla_cilow_obj", index=False)
    table_cilow_time.to_excel(writer, sheet_name="tabla_cilow_time", index=False)
    table_cihigh_inc.to_excel(writer, sheet_name="tabla_cihigh_inc", index=False)
    table_cihigh_obj.to_excel(writer, sheet_name="tabla_cihigh_obj", index=False)
    table_cihigh_time.to_excel(writer, sheet_name="tabla_cihigh_time", index=False)
    table_std_inc.to_excel(writer, sheet_name="tabla_std_inc", index=False)
    table_std_obj.to_excel(writer, sheet_name="tabla_std_obj", index=False)
    table_std_time.to_excel(writer, sheet_name="tabla_std_time", index=False)
    table_median_inc.to_excel(writer, sheet_name="tabla_median_inc", index=False)
    table_median_obj.to_excel(writer, sheet_name="tabla_median_obj", index=False)
    table_median_time.to_excel(writer, sheet_name="tabla_median_time", index=False)
    table_iqr_inc.to_excel(writer, sheet_name="tabla_iqr_inc", index=False)
    table_iqr_obj.to_excel(writer, sheet_name="tabla_iqr_obj", index=False)
    table_iqr_time.to_excel(writer, sheet_name="tabla_iqr_time", index=False)


files.download("ExperimentosComputacionalesSP.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>